In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from ipynb.fs.full.IO_utils import read_imgs_dir
from ipynb.fs.full.Show_utils import plot_query_retrieval

import keras
import cv2 
import glob 


Using TensorFlow backend.


In [3]:
modelName = "vgg19"
trainModel = True
parallel = True  # for using multicore processing


In [4]:
# Make paths
dataTrainDir = os.path.join(os.getcwd(), "train")
dataTestDir = os.path.join(os.getcwd(), "test")
outDir = os.path.join(os.getcwd(), "output", modelName)
if not os.path.exists(outDir):
    os.makedirs(outDir)

In [5]:
#reading images of jpg or jpeg format
extensions = [".jpg", ".jpeg"]
print("read train images '{}'...".format(dataTrainDir))
imgs_train = read_imgs_dir(dataTrainDir, extensions, parallel=parallel)
print("read test images '{}'...".format(dataTestDir))
imgs_test = read_imgs_dir(dataTestDir, extensions, parallel=parallel)
shape_img = imgs_train[0].shape
print("image size = {}".format(shape_img))

read train images 'C:\Users\ashu\Documents\jupterNotebook\avantari_task\avantari_jupyter\train'...
read test images 'C:\Users\ashu\Documents\jupterNotebook\avantari_task\avantari_jupyter\test'...
image size = (512, 512, 3)


In [6]:
#Use the bottleneck features of a pre-trained network vgg19 and remove its last layers
if modelName in ["vgg19"]:

    # Load pre-trained VGG19 model + higher level layers
    print("Loading VGG19 pre-trained model")
    model = tf.keras.applications.VGG19(weights='imagenet', include_top=False,
                                        input_shape=shape_img)
    model.summary()

    shape_img_resize = tuple([int(x) for x in model.input.shape[1:]])
    input_shape_model = tuple([int(x) for x in model.input.shape[1:]])
    output_shape_model = tuple([int(x) for x in model.output.shape[1:]])
    n_epochs = None

else:
    raise Exception("Invalid modelName!")


Loading VGG19 pre-trained model
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   

In [7]:
# Print model info
print("input_shape_model = {}".format(input_shape_model))
print("output_shape_model = {}".format(output_shape_model))


input_shape_model = (512, 512, 3)
output_shape_model = (16, 16, 512)


In [8]:
#function to extract feature form images using vgg19 model and resize images to 512 x 512
#Convert our image database into feature vectors using VGG model
def extract_vector(path):
    vgg_feature_list = []

    for im in glob.glob(path):

        im = cv2.imread(im)
        im = cv2.resize(im,(512,512))
        img = tf.keras.applications.vgg19.preprocess_input(np.expand_dims(im.copy(), axis=0))
        vgg_feature = model.predict(img)
        vgg_feature_np = np.array(vgg_feature)
        vgg_feature_list.append(vgg_feature_np.flatten())

    return np.array(vgg_feature_list)

In [9]:
x = extract_vector('train/*')


In [10]:
y = extract_vector('test/*')

In [11]:
x.shape

(4738, 131072)

In [12]:
# Fit kNN model on training images
# Compute similarities between our image feature vectors using an inner-product such as cosine similarity or euclidean distance
print("Fitting k-nearest-neighbour model on training images...")
knn = NearestNeighbors(n_neighbors=15, metric="cosine")
knn.fit(x)

Fitting k-nearest-neighbour model on training images...


NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=15, p=2, radius=1.0)

In [13]:
#For each image, select the images with the top-k similarity scores and show it and print output in "output" folder
print("Performing image retrieval on test images...")
for i, emb_flatten in enumerate(y):
    _, indices = knn.kneighbors([emb_flatten]) # find k nearest train neighbours
    img_query = imgs_test[i] # query image
    imgs_retrieval = [imgs_train[idx] for idx in indices.flatten()] # retrieval images
    outFile = os.path.join(outDir, "{}_retrieval_{}.png".format(modelName, i))
    plot_query_retrieval(img_query, imgs_retrieval, outFile)

Performing image retrieval on test images...
